# Tutorial 07: Bayesian optimization



In [ ]:
import numpy as np
import pandas as pd
import torch
import ultralytics
import time

In [ ]:
import electricmayhem.whitebox as em

In [ ]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench','bird', 'cat',
                'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
                'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
                'clock', 'vase', 'scissors', 'teddy bear', 'hair drier','toothbrush']

## create

Let's do color patches this time, but use a soft proofer during training to make sure the colors are realistic

In [ ]:
tile_size = 256
patch_size = 64

In [ ]:
tiler = em.PatchTiler({"ground":(tile_size, tile_size)})

In [ ]:
proofer = em.SoftProofer("data/profile.icc")

## implant

Reuse the same target dataset from tutorial 01.

In [ ]:
labels = pd.read_csv("data/toycar/toycar_warp_dataset.csv")
labels = labels[labels.patch == "ground"]
len(labels)

In [ ]:
labels.head()

Names of the 3 patches we'll train:

In [ ]:
labels.patch.unique()

The `em.WarpPatchImplanter()` class will take care of differentiably deforming and implanting patches (with kornia doing most of the heavy lifting). We need two inputs:

* the `DataFrame` of target labels
* a dictionary of patch shapes (at the point of implanting, so they'll be 3-channel); the implanter will use this to precompute transformation matrices

In [ ]:
patch_shapes = {k:(3,patch_size, patch_size) for k in ['ground']}
imp = em.WarpPatchImplanter(labels, patch_shapes=patch_shapes, dataset_name="toycar_warp_only_ground")

## compose

The main tool `electricmayhem` has so far is `em.KorniaAugmentationPipeline()`, which just wraps the `kornia.augmentation` API. Initialize it with a dictionary of image augmentations, where each value is the keyword arguments that augmentation takes.

In [ ]:
aug = em.KorniaAugmentationPipeline({"ColorJiggle":{"brightness":0.2, "contrast":0.2, "hue":0.1, "saturation":0.1},
                                    "RandomAffine":{"scale":(0.9,1.1), "shear":10, "padding_mode":"reflection", "degrees":0}})

## infer

Here's where we'll depart from tutorial 01. Let's train a patch using two YOLOv8 models and test performance on a YOLOv11.

In [ ]:
yolov8n = ultralytics.YOLO("yolov8n.pt").model.eval()

Pass dictionaries to `em.YOLOWrapper` to associate each model with a name (to make sure our logs are interpretable) as well as a YOLO version. In this case it won't matter because output formats of v8 and v11 are the same.

In [ ]:
yolo = em.YOLOWrapper(yolov8n, yolo_version=8, classnames=COCO_CLASSES, iouthresh=1.)

## assemble the pipeline

Take all of the steps we built above and assemble into a `Pipeline` object:

In [ ]:
pipeline = tiler+proofer+imp+aug+yolo

## Write a loss function

Note that in this case, success is when the patch is detected **above** 0.25 instead of below

In [ ]:
threshold = 0.3

def loss(output, **kwargs):
    maxdetect_boxes = output[0][:,:,4] # (batch, num_boxes)
    maxdetect = torch.max(maxdetect_boxes, 1)[0]  # (batch,)

    inverse_maxdetect = torch.mean(1-maxdetect_boxes, -1)
    hard_threshold = torch.mean(1 - torch.minimum(maxdetect_boxes, torch.tensor(threshold)), -1)    

    # how many boxes per image above the default detection threshold?
    boxcount = torch.sum((maxdetect_boxes >= 0.25).type(torch.float32), -1)

    with torch.no_grad():
        detects = output[0].permute(0,2,1) # (batch, 5+num_classes, num_boxes)
        detects = torch.concatenate([detects[:,:4,:], detects[:,5:,:]],1) # (batch, 4+num_classes, num_boxes)
        t0 = time.time()
        nms = ultralytics.utils.ops.non_max_suppression(detects, conf_thres=0.1)
        t1 = time.time()

    outdict = {
        "inverse_maxdetect":inverse_maxdetect,
        "hard_threshold":hard_threshold,
        "boxcount":boxcount,
        "nms_time":(t1-t0)*torch.ones_like(maxdetect)
    }
    return outdict

Pass the loss function to your pipeline along with a dictionary giving the shapes of a batch of test patches, so it can check the inputs/outputs before you start training:

In [ ]:
pipeline.set_loss(loss, test_patch_shape={k:(2,3,patch_size, patch_size) for k in ['ground']})

## Train the patch

When we set logging- we can also add arbitrary key-value pairs two ways as keyword arguments to `pipeline.set_logging()`:

* `extra_params` will add them as MLFlow parameters; this is useful for tracking exogenous variables when your pipeline is part of a larger experiment
* `tags` will add them to as MLFlow tags

In [ ]:
#pipeline.set_logging(logdir="logs_07/08",
#                    mlflow_uri="http://127.0.0.1:5000",
#                    experiment_name="electricmayhem_tutorial_07_bayesian_optimization",
#                    extra_params={"tile_size":tile_size, "pach_size":patch_size})

Second, explicitly tell it to initialize the patches. If you want you could alternatively pass it a dictionary of patches pre-initialized to whatever you want.

In [ ]:
#pipeline.initialize_patch_params(patch_shape={k:(3,patch_size, patch_size) for k in ['ground']})

All of our classes inherit from `torch.nn.Module` so this should look familiar:

In [ ]:
pipeline.cuda();

In [ ]:
pipeline.optimize(
    "nms_time",
    "logs_latency_attack/",
    {"ground":(3,patch_size, patch_size)},
    1000,
    2500,
    24,
    num_eval_steps=100,
    mlflow_uri="http://127.0.0.1:5000",
    experiment_name="electricmayhem_tutorial_07_bayesian_optimization_2",
    extra_params={"tile_size":tile_size, "pach_size":patch_size},
    minimize=False,
    learning_rate=(1e-4, 1e-1, "log"),
    lr_decay="cosine",
    optimizer=["adam", "mifgsm"],
    inverse_maxdetect=(0,1),
    hard_threshold=(0,1)
)

When training the patch- the loss function will return two `maxdetect` terms, one for each model, so we'll need to specify weights for each explicitly:

In [ ]:
import ultralytics

In [ ]:
ultralytics.utils.ops.non_max_suppression?

In [ ]:
#ultralytics.utils.ops.non_max_suppression?

In [ ]:
patch

In [ ]:
patch["ground"].shape

In [ ]:
foo, _ = tiler({"ground":patch["ground"].unsqueeze(0)}, evaluate=True)

In [ ]:
em.plot(patch["ground"])

In [ ]:
em.plot(foo["ground"])

In [ ]:
foo["ground"].shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def sig(x):
    return 1/(1+np.exp(-x))

In [ ]:
x = np.linspace(-10,10,100)
plt.plot(x, sig(x));

In [ ]:
torch.minimum?